In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import networkx as nx
import glob
import re

from Functions import get_distance_matrix

In [3]:
#use glob glob to get all unordered files
CSV_filenames = glob.glob("DATA/Networks/1_Monthly_MAX_MST_Threshold_Networks/*.csv")

#Access the node table for attributes
stock_attribute_df = pd.read_csv("DATA/Stock_Data/NODE_TABLE.csv") 



# Extract the numerical part of each filename and sort the list in numerical order
CSV_filenames.sort(key=lambda x: int(re.search(r'(\d+)', x.split('_')[-1]).group()))

#How to extract the date from the filename
Date = CSV_filenames[-1].split("\\")[-1].split("_")[-2]
Date



'30-09-2021'

In [4]:

#Cell is work in progress for later?

#Extracting tickers and making a dataframe template

Uncleaned_Data_Price = pd.read_csv("DATA/Stock_Data/stock_log_returns_2Y.csv", index_col='Date')

cleaned_Data_Price = Uncleaned_Data_Price.dropna(axis=1)

Tickers = list(cleaned_Data_Price.columns)

#Make a df where the first column are the Tickers
stock_cluster_tracker = pd.DataFrame(Tickers, columns=["Tickers"])


# Running the communities

In [ ]:

largest_cluster_stocks_df = pd.DataFrame(columns=["Date", "Stocks"])
Second_largest_cluster_stocks_df = pd.DataFrame(columns=["Date", "Stocks"])
Smallest_cluster_stocks_df = pd.DataFrame(columns=["Date", "Stocks"])



silhouette_score_tracker = []
CSV_list = []

b = 0

largest_cluster_stocks_df = pd.DataFrame(columns=["Date", "Stocks"])
Second_largest_cluster_stocks_df = pd.DataFrame(columns=["Date", "Stocks"])
Smallest_cluster_stocks_df = pd.DataFrame(columns=["Date", "Stocks"])




for i, file_path in enumerate(CSV_filenames):

    Date = file_path.split("\\")[-1].split("_")[-2]

    

    Network_Matrix = pd.read_csv(file_path, index_col=0)
    adjacency_matrix = Network_Matrix.to_numpy()
    G = nx.from_pandas_adjacency(Network_Matrix)





    ##### Clustering starts####

    cluster_assignments = nx.community.louvain_communities(G, seed=42)            #resolution changing #function changing   ###, resolution=1
    
    modularity = nx.community.quality.modularity(G, cluster_assignments)


    #somthing odd with the below#####   ---- add to an empty df, (it had values in stock_cluster_tracker)


    stock_cluster_tracker[Date] = np.nan

    for cluster_id, stocks_names_in_cluster in enumerate(cluster_assignments):
        for stock_name in stocks_names_in_cluster:
            stock_cluster_tracker.loc[stock_name, Date] = cluster_id + 1
    


    #find optimal resolution
    # for i in range(101, 110, 50):
    #     print(f"resoultion: {i/100}", end="\t| ")
    #     output = nx.community.louvain_communities(G, resolution=i/100, seed=42)
    #     print(f"Total Clusters: {len(output)}", end="\t | ")
    #     modularity = nx.community.quality.modularity(G, output)
    #     print(f"Modularity Score: {modularity}")

    stock_cluster_date_count_stats = stock_cluster_tracker[Date].value_counts()

    total_cluster_size = stock_cluster_date_count_stats.sum()



    #Laregst cluster
    largest_cluster_id = stock_cluster_date_count_stats.index[0]
    stocks_in_largest_cluster = stock_cluster_tracker[stock_cluster_tracker[Date] == largest_cluster_id].index.tolist()
    # Creating new row for concatenating
    new_row_largest_cluster_stocks_df = pd.DataFrame([{"Date": Date, "Stocks": stocks_in_largest_cluster}])
    largest_cluster_stocks_df = pd.concat([largest_cluster_stocks_df, new_row_largest_cluster_stocks_df], ignore_index=True)

    #Second Laregst cluster
    Second_largest_cluster_id = stock_cluster_date_count_stats.index[1]
    stocks_in_Second_largest_cluster = stock_cluster_tracker[stock_cluster_tracker[Date] == Second_largest_cluster_id].index.tolist()
    # Creating new row for concatenating
    new_row_Second_largest_cluster_stocks_df = pd.DataFrame([{"Date": Date, "Stocks": stocks_in_Second_largest_cluster}])
    Second_largest_cluster_stocks_df = pd.concat([Second_largest_cluster_stocks_df, new_row_Second_largest_cluster_stocks_df], ignore_index=True)

    #Smallest cluster
    Smallest_cluster_id = stock_cluster_date_count_stats.index[-1]
    stocks_in_Smallest_cluster = stock_cluster_tracker[stock_cluster_tracker[Date] == Smallest_cluster_id].index.tolist()

    # Creating new row for concatenating
    new_row_Smallest_cluster_stocks_df = pd.DataFrame([{"Date": Date, "Stocks": stocks_in_Smallest_cluster}])
    Smallest_cluster_stocks_df = pd.concat([Smallest_cluster_stocks_df, new_row_Smallest_cluster_stocks_df], ignore_index=True)

    
    CSV_list.append({
        "Date": Date,
        "Number of clusters": len(cluster_assignments),
        "Modularity": modularity,
        "Average cluster size": total_cluster_size/len(cluster_assignments),
        "Largest Cluster ID" : stock_cluster_date_count_stats.index[0], #first row and first columns aka cluster with the highest stocks .index is cluster id
        "Largest cluster Size": stock_cluster_date_count_stats.iloc[0],
        "Smallest Cluster ID": stock_cluster_date_count_stats.index[-1],
        "Smallest cluster Size": stock_cluster_date_count_stats.iloc[-1],

        "Three_smallest_cluster_id" : list(stock_cluster_date_count_stats.index[-3:]),
        "Three_smallest_cluster_stock_count" : list(stock_cluster_date_count_stats.iloc[-3:]),

        "Three_largest_cluster_id" : list(stock_cluster_date_count_stats.index[:3]),
        "Three_largest_cluster_stock_count" : list(stock_cluster_date_count_stats.iloc[:3]),
        "num_connected_components": num_connected_components

        #"clusters" : clusters,
        #"Ordered cluster count ID": stock_cluster_tracker.value_counts()
      })
    
    b=b+1

    print(f"{b/(Uncleaned_Data_Price.shape[0]-21)*100} % down ") #{int(Final_Increment) - b} to go
    # if i == 3:
    #     break

transposed_df_largest_cluster_stocks_df= largest_cluster_stocks_df.set_index("Date")["Stocks"].apply(pd.Series).T
transposed_df_largest_cluster_stocks_df.to_csv("DATA/Network_Stats/Cluster_Stats/1_MAX_MST/Largest_Cluster_Stocks.csv", index = False)

transposed_Second_largest_cluster_stocks_df = Second_largest_cluster_stocks_df.set_index("Date")["Stocks"].apply(pd.Series).T
transposed_Second_largest_cluster_stocks_df.to_csv("DATA/Network_Stats/Cluster_Stats/1_MAX_MST/Second_largest_cluster_stocks.csv", index = False)

transposed_Smallest_cluster_stocks_df = Smallest_cluster_stocks_df.set_index("Date")["Stocks"].apply(pd.Series).T
transposed_Smallest_cluster_stocks_df.to_csv("DATA/Network_Stats/Cluster_Stats/1_MAX_MST/Smallest_cluster_stocks.csv", index = False)

    
stock_cluster_tracker.to_csv("DATA/Network_Stats/Cluster_Stats/1_MAX_MST/Clusters_stock_tracker_louvain.csv")

CSV_list_df = pd.DataFrame(CSV_list)
CSV_list_df.to_csv("DATA/Network_Stats/Cluster_Stats/1_MAX_MST/Louvain_statistics.csv")

print(Date)

0.2066115702479339 % down 
0.4132231404958678 % down 
0.6198347107438017 % down 
0.8264462809917356 % down 
1.0330578512396695 % down 
1.2396694214876034 % down 
1.4462809917355373 % down 
1.6528925619834711 % down 
1.859504132231405 % down 
2.066115702479339 % down 
2.272727272727273 % down 
2.479338842975207 % down 
2.6859504132231407 % down 
2.8925619834710745 % down 
3.0991735537190084 % down 
3.3057851239669422 % down 
3.512396694214876 % down 
3.71900826446281 % down 
3.925619834710744 % down 
4.132231404958678 % down 
4.338842975206612 % down 
4.545454545454546 % down 
4.75206611570248 % down 
4.958677685950414 % down 
5.1652892561983474 % down 
5.371900826446281 % down 
5.578512396694215 % down 
5.785123966942149 % down 
5.991735537190083 % down 
6.198347107438017 % down 
6.40495867768595 % down 
6.6115702479338845 % down 
6.8181818181818175 % down 
7.024793388429752 % down 
7.231404958677685 % down 
7.43801652892562 % down 
7.644628099173553 % down 
7.851239669421488 % down 
8

C:\Users\chris\AppData\Local\Temp\ipykernel_11956\1492828742.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_cluster_tracker[Date] = np.nan


20.66115702479339 % down 


C:\Users\chris\AppData\Local\Temp\ipykernel_11956\1492828742.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_cluster_tracker[Date] = np.nan


20.867768595041323 % down 


...

99.58677685950413 % down 


C:\Users\chris\AppData\Local\Temp\ipykernel_11956\1492828742.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_cluster_tracker[Date] = np.nan


99.79338842975206 % down 


C:\Users\chris\AppData\Local\Temp\ipykernel_11956\1492828742.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_cluster_tracker[Date] = np.nan


100.0 % down 
30-09-2021


# Highest density community > 50  stocks to avoid highly connected insignificantly small communities.

In [39]:

Densely_cluster_stocks_df = pd.DataFrame(columns=["Date", "Stocks"])
Desnisty_Densely_cluster_stocks_df = []

silhouette_score_tracker = []
CSV_list = []
b = 0

for i, file_path in enumerate(CSV_filenames):

    #Read the networks
    Date = file_path.split("\\")[-1].split("_")[-2]
    print(Date)
    Network_Matrix = pd.read_csv(file_path, index_col=0)
    adjacency_matrix = Network_Matrix.to_numpy()
    G  = nx.from_pandas_adjacency(Network_Matrix)

    ##### Clustering starts####
    cluster_assignments = nx.community.louvain_communities(G, seed=42)          
    modularity = nx.community.quality.modularity(G, cluster_assignments)
    stock_cluster_tracker[Date] = np.nan


    Highest_density_n_cluster = [0,0]
    for cluster_id, stocks_names_in_cluster in enumerate(cluster_assignments):

        Minimum_cluster_size = 50
        
        # #Find densely connected clusters
        if len(stocks_names_in_cluster) >= Minimum_cluster_size:
            
            cluster_subgraph = G.subgraph(stocks_names_in_cluster)
            cluster_subgraph_density = nx.density(cluster_subgraph)

            #Iterating to find the highest density cluster
            if cluster_subgraph_density > Highest_density_n_cluster[0]:
                Highest_density_n_cluster[0] = cluster_subgraph_density
                Highest_density_n_cluster[1] = list(stocks_names_in_cluster)

        #Assign stocks to their cluster
        for stock_name in stocks_names_in_cluster:
            stock_cluster_tracker.loc[stock_name, Date] = cluster_id + 1
    
    
    #print(Highest_density_n_cluster)

    Stocks_in_most_dense_cluster = Highest_density_n_cluster[1]
    Density_Highest_Density_Cluster = Highest_density_n_cluster[0]

    #Laregst cluster
    
    # Creating new row for concatenating
    new_row_dense_cluster_stocks_df = pd.DataFrame([{"Date": Date, "Stocks": Stocks_in_most_dense_cluster}])
    Densely_cluster_stocks_df = pd.concat([Densely_cluster_stocks_df, new_row_dense_cluster_stocks_df], ignore_index=True)
    #print(dense_cluster_stocks_df)

    #Density only
    Desnisty_Densely_cluster_stocks_df.append({
        "Date": Date,
        "Density_Highest_Density_Cluster": Density_Highest_Density_Cluster
    })

    b=b+1

    print(f"{b/(Uncleaned_Data_Price.shape[0]-21)*100} % down ") #{int(Final_Increment) - b} to go
    # if i == 5:
    #     break

transposed_df_dense_cluster_stocks_df= Densely_cluster_stocks_df.set_index("Date")["Stocks"].apply(pd.Series).T
transposed_df_dense_cluster_stocks_df.to_csv("DATA/Network_Stats/Cluster_Stats/1_MAX_MST/Highest_Density_GT50_Cluster_Stocks.csv", index = False)

Desnisty_Densely_cluster_stocks_df = pd.DataFrame(Desnisty_Densely_cluster_stocks_df)
Desnisty_Densely_cluster_stocks_df.to_csv("DATA/Network_Stats/Cluster_Stats/1_MAX_MST/Density_Highest_Density_GT50_Cluster.csv", index=False)

print(transposed_df_dense_cluster_stocks_df)

30-10-2019
0.2066115702479339 % down 
31-10-2019
0.4132231404958678 % down 
01-11-2019
0.6198347107438017 % down 
04-11-2019
0.8264462809917356 % down 
05-11-2019
1.0330578512396695 % down 
06-11-2019
1.2396694214876034 % down 
07-11-2019
1.4462809917355373 % down 
08-11-2019
1.6528925619834711 % down 
11-11-2019
1.859504132231405 % down 
12-11-2019
2.066115702479339 % down 
13-11-2019
2.272727272727273 % down 
14-11-2019
2.479338842975207 % down 
15-11-2019
2.6859504132231407 % down 
18-11-2019
2.8925619834710745 % down 
19-11-2019
3.0991735537190084 % down 
20-11-2019
3.3057851239669422 % down 
21-11-2019
3.512396694214876 % down 
22-11-2019
3.71900826446281 % down 
25-11-2019
3.925619834710744 % down 
26-11-2019
4.132231404958678 % down 
27-11-2019
4.338842975206612 % down 
29-11-2019
4.545454545454546 % down 
02-12-2019
4.75206611570248 % down 
03-12-2019
4.958677685950414 % down 
04-12-2019
5.1652892561983474 % down 
05-12-2019
5.371900826446281 % down 
06-12-2019
5.57851239669421

# Visulising the results

In [5]:
Comminity_stats_df = pd.read_csv("DATA/Network_Stats/Cluster_Stats/1_MAX_MST/Louvain_statistics.csv")
Comminity_stats_df

,Unnamed: 0,Date,Number of clusters,Modularity,Average cluster size,Largest Cluster ID,Largest cluster Size,Smallest Cluster ID,Smallest cluster Size,Three_smallest_cluster_id,Three_smallest_cluster_stock_count,Three_largest_cluster_id,Three_largest_cluster_stock_count,num_connected_components
0,0,30-10-2019,7,0.288724,348.285714,6.0,846,7.0,43,"[2.0, 3.0, 7.0]","[247, 101, 43]","[6.0, 1.0, 5.0]","[846, 445, 403]",1
1,1,31-10-2019,9,0.353131,270.888889,7.0,454,8.0,77,"[9.0, 3.0, 8.0]","[167, 97, 77]","[7.0, 4.0, 5.0]","[454, 393, 377]",1
2,2,01-11-2019,9,0.346534,271.111111,3.0,622,7.0,110,"[5.0, 6.0, 7.0]","[141, 119, 110]","[3.0, 2.0, 9.0]","[622, 450, 405]",1
3,3,04-11-2019,9,0.315140,270.888889,2.0,618,1.0,53,"[5.0, 4.0, 1.0]","[101, 90, 53]","[2.0, 8.0, 9.0]","[618, 466, 445]",1
4,4,05-11-2019,10,0.313849,244.200000,6.0,498,7.0,69,"[5.0, 3.0, 7.0]","[171, 118, 69]","[6.0, 10.0, 2.0]","[498, 415, 376]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,479,24-09-2021,5,0.210831,491.000000,3.0,1163,5.0,13,"[2.0, 4.0, 5.0]","[502, 14, 13]","[3.0, 1.0, 2.0]","[1163, 763, 502]",1
480,480,27-09-2021,3,0.221945,818.000000,2.0,963,3.0,603,"[2.0, 1.0, 3.0]","[963, 888, 603]","[2.0, 1.0, 3.0]","[963, 888, 603]",1
481,481,28-09-2021,4,0.240383,613.500000,3.0,980,2.0,23,"[1.0, 4.0, 2.0]","[878, 573, 23]","[3.0, 1.0, 4.0]","[980, 878, 573]",2
482,482,29-09-2021,5,0.268681,490.800000,2.0,926,1.0,2,"[5.0, 4.0, 1.0]","[567, 102, 2]","[2.0, 3.0, 5.0]","[926, 857, 567]",2


In [9]:
#######################################################################################--------------------------------------
# OUTPUT
SAVE_FIGURE_DIRECTORY = "DATA/Figures/Cluster_Figs/1_MAX_MST" # '.' dot means current directory
SAVE_FIGURE_FILE_NAME = ""

# INPUT
CSV_FILE_PATH = 'Data/Network_Stats/Cluster_Stats/1_MAX_MST/Louvain_statistics.csv'  

LABEL_FONT_SIZE = 25
X_AXIS_df_column_name = "Date"
Y_AXIS_df_column_name = 'Modularity'
X_AXIS_GRAPH_LABEL = X_AXIS_df_column_name # replace it with a string, e,g "my x-axis" 
Y_AXIS_GRAPH_LABEL = Y_AXIS_df_column_name 
############################################################################################################-------------------

df_csv = pd.read_csv(CSV_FILE_PATH)    #dataframe of csv

#Must match date format!!!!!!!!!
df_csv["Date"] = pd.to_datetime(df_csv["Date"], format="%d-%m-%Y")
 # Ensure the date column is in datetime format

#csv_column_names.remove("Date")



Select the columns you want to plot

In [10]:
csv_column_names = ["Number of clusters",'Modularity','Average cluster size', "Largest cluster Size"]
csv_column_names

##If you want to select all
#csv_column_names = df_csv.columns.tolist()

['Number of clusters',
 'Modularity',
 'Average cluster size',
 'Largest cluster Size']

In [12]:
plt.rcParams['font.size'] = 18          #change the font size!!
sns.set_style("whitegrid")                                                      # comment this out if the graph looks bad
for column_name in csv_column_names:
    print(f"Delete me later, column: {column_name}")
    if column_name.endswith("Stock"):
        print(f"\tSkipping column: {column_name}")
        continue
    
    plt.figure(figsize=(15, 8.3), dpi=1500)                                     # change graph size here: figsize(width, height), dpi controls the quality of the image
    sns.lineplot(data=df_csv, x = X_AXIS_df_column_name, y = column_name, color='#00008b', linewidth='1.87') 

    plt.ylabel(column_name, fontsize = LABEL_FONT_SIZE)
    plt.xlabel(X_AXIS_GRAPH_LABEL, fontsize= LABEL_FONT_SIZE)

    

    plt.grid(visible=True)
    plt.xticks(rotation = 80)
    plt.xlim(mdates.date2num(df_csv[X_AXIS_df_column_name].min()), mdates.date2num(df_csv[X_AXIS_df_column_name].max()))

    # Additional graph formatting, comment it out if producing error 

    # # Set major locator and formatter for x-axis
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3)) # x-values are skipped every (interval=)3 month 
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%b-%Y"))#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d/%m/%Y")) # don't change this

    # # # Add a grid line for a specific interval (e.g., every 6 months)
    plt.gca().xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
    plt.grid(which='minor', linestyle='-', linewidth='0.7', color='lightgrey')



    #####--------------############---------------###########----------#########-
    plt.axvspan(df_csv["Date"][75], df_csv["Date"][98], color='lightgrey')            #THIS IS FOR COVID GREY AREA!

    plt.tight_layout()
    plt.savefig(f"{SAVE_FIGURE_DIRECTORY}/{SAVE_FIGURE_FILE_NAME}{column_name}.pdf", dpi=1500)
    plt.clf()

Delete me later, column: Number of clusters
Delete me later, column: Modularity
Delete me later, column: Average cluster size
Delete me later, column: Largest cluster Size


<Figure size 22500x12450 with 0 Axes>

<Figure size 22500x12450 with 0 Axes>

<Figure size 22500x12450 with 0 Axes>

<Figure size 22500x12450 with 0 Axes>